### Solutions to the problems of Chapter 9
## Priority Queues and Heaps

### helper classes 

In [1]:
class Empty(Exception):
    pass 
class _DoublyLinkedBase:
    class _Node:
        __slots__='_element', '_prev', '_next'
        
        def __init__(self, element, prev, nxt):
            self._element = element 
            self._prev = prev
            self._next = nxt 
            
    def __init__(self):
        self._header = self._Node(None, None, None)
        self._trailer = self._Node(None, None, None)
        self._header._next = self._trailer 
        self._trailer._prev = self._header
        self._size = 0 
        
    def __len__(self): 
        return self._size 
    
    def is_empty(self):
        return self._size == 0 
    
    def _insert_between(self, e, predecessor, successor):
        newest = self._Node(e, predecessor, successor)
        predecessor._next = newest
        successor._prev = newest 
        self._size += 1
        return newest 
    
    def _delete_node(self, node):
        predecessor = node._prev 
        successor = node._next 
        predecessor._next = successor 
        successor._prev = predecessor 
        self._size -= 1
        element = node._element 
        node._prev = node._next = node._element = None
        return element 

class PositionalList(_DoublyLinkedBase):
    def __init__(self):
        super().__init__() 
        
    class Position: 
        def __init__(self, container, node):
            self._container = container 
            self._node = node 
            
        def element(self):
            return self._node._element 
        def __eq__(self, other): 
            return type(other) is type(self) and other._node is self._node
        def __ne__(self, other):
            return not(self==other)
    
    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper position type')
        if p._container is not self:
            raise ValueError('p does not belong to this type')
        if p._node._next is None:
            raise ValueError('p is no longer valid')
        return p._node 
    
    def _make_position(self, node):
        if (node is self._header) or (node is self._trailer):
            return None
        else:
            return self.Position(self, node)
        
    def first(self):
        return self._make_position(self._header._next)
    
    def last(self):
        return self._make_position(self._trailer._prev)
    
    def before(self, p):
        node = self._validate(p)
        return self._make_position(node._prev)
    
    def after(self, p):
        node = self._validate(p)
        return self._make_position(node._next)
    
    def __iter__(self):
        cursor = self.first() 
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)
            
    def _insert_between(self, e, predecessor, successor):
        node = super()._insert_between(e, predecessor, successor)
        return self._make_position(node)
    
    def add_first(self, e):
        return self._insert_between(e, self._header, self._header._next)
    
    def add_last(self, e):
        return self._insert_between(e, self._trailer._prev, self._trailer)
    
    def add_before(self, p, e):
        original = self._validate(p)
        return self._insert_between(e, original._prev, original)
    
    def add_after(self, p, e):
        original = self._validate(p)
        return self._insert_between(e, original, original._next)
    
    def delete(self, p):
        original = self._validate(p)
        return self._delete_node(original)
    
    def replace(self, p, e):
        original = self._validate(p)
        old_value = original._element 
        original._element = e
        return old_value

### Implementation to the Priority Queue ADT,  UnsortedPriorityQueue   

In [2]:
class PriorityQueueBase:
    class _Item:
        __slots__= '_key', '_value'
        def __init__(self, key, value):
            self._key = key
            self._value = value 
        def __lt__(self, other):
            return self._key < other._key 
    def is_empty(self):
        return len(self) ==0 

In [3]:
class UnsortedPriorityQueue(PriorityQueueBase): 
    def __init__(self):
        self._data = PositionalList() 
    
    def __len__(self):
        return len(self._data)
    
    def add(self, key, value):
        self._data.add_last(self._Item(key, value))
    
    def _find_min(self):
        if self.is_empty():
            raise Empty("Empty Prioirty Queue")
        minimum = self._data.first() 
        current = self._data.after(minimum)
        while current is not None:
            if current.element()< minimum.element():
                minimum = current
            current = self._data.after(current)
        return minimum 
    
    def min(self):
        item = self._find_min().element() 
        return (item._key, item._value) 
    
    def remove_min(self):
        minimum = self._find_min()
        item = self._data.delete(minimum) 
        return (item._key, item._value) 

In [4]:
class SortedPriorityQueue():
    def __init__(self):
        self._data = PositionalList() 
    def __len__(self):
        return len(self._data)
    def add(self, key, value):
        new = self._Item(key, value)
        current = self._data.last() 
        while current is not None and new<current.element(): 
            current = self._data.before(current)
        if current is None: 
            self._data.add_first(new)
        else:
            self._data.add_after(current, new)
            
    def min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue")
        min_item = self._data.first().element()
        return (min_item._key, min_item._value)
    
    def remove_min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue")
        min_item = self._data.delete(self._data.first())
        return (min_item._key, min_item._value)

## Heap Data strcutre 

In [6]:
class HeapPriorityQueue(PriorityQueueBase):
    def __init__(self, contents = ()): 
        self._data = [self._Item(k, v) for k, v in contents]
        if len(self._data)>1:
            self._heapify() 
            
    def __len__(self):
        return len(self._data) 
    
    def _heapify(self): 
        start = self.parent(len(self._data)-1)  
        for j in range(start, -1, -1):
            self._downheap(j)
    
    def add(self, key, value):
        self._data.append(self._Item(key, value))
        self._upheap(len(self._data)-1)
    
    def min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue!")
        mini = self._data[0]
        return (mini._key, mini._value)
    
    def remove_min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue!")
        self._swap(0, len(self._data)-1)
        mini = self._data.pop()
        self._downheap(0)
        return (mini._key, mini._value) 
    
    def _parent(self, j):
        return (j-1)//2 
    
    def _left(self, j):
        return (2*j) + 1 
    
    def _right(self, j):
        (2*j) + 2 
    
    def _has_left(self, j):
        return self._left(j) < len(self._data)
    
    def _has_right(self, j):
        return self._right(j) < len(self._data)
    
    def _swap(self, i, j):
        self._data[i], self._data[j] = self._data[j], self._data[i]
    
    def _uphead(self, j):
        parent = self._parent(j)
        if j>0 and (self._data[j]<self._data[parent]): 
            self._swap(j, parent)
            self._uphead(parent)
    
    def _downheap(self, j):
        if self._has_left(j):
            small_child = self._left(j)
            if self._has_right(j):
                right_child = self._right(j)
                if self._data[right_child]<self._data[small_child]:
                    small_child = right_child
            
            if self._data[small_child]<self._data[j]: 
                self._swap(j, small_child)
                self._downheap(small_child)